# Library imports, configurations

In [4]:
import os
import re
import s3fs
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from unidecode import unidecode
from wordcloud import WordCloud
import nltk
nltk.download('punkt')

from stop_words import get_stop_words
import unicodedata

from nltk.tokenize import word_tokenize
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import connections

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [7]:
BUCKET = "jplaton/diffusion"
fs.ls(BUCKET)

['jplaton/diffusion/.keep',
 'jplaton/diffusion/ted',
 'jplaton/diffusion/visio_mel']

In [8]:
BUCKET_CLEM = "clementg/diffusion"
fs.ls(BUCKET_CLEM)

['clementg/diffusion/.keep',
 'clementg/diffusion/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv']

# Functions

In [80]:
# Suppression des accents

def remove_accent(s):

    s = unidecode(str(s))
    return(s)

# Suppression de la ponctuation

def remove_punctuation(s):
    # On retire la ponctuation
 
    s = re.sub(r'[^A-Za-z0-9]',' ',str(s))
    return(s)

# Réduction de la casse

def tolower(s):
    s = ' '.join(str(s).lower().split())
    return(s)


# Sur cette base, on crée une petite fonction qui retire les stop words
stopwords = get_stop_words('french')   
stopwords = [unicodedata.normalize('NFKD',m).encode('ASCII', 'ignore').decode() for m in stopwords]

def remove_stopwords(s):
    s = [w for w in word_tokenize(s) if w not in stopwords]
    s = ' '.join(s)
    return(s)

def remove_accent_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
    return df_c

def remove_punctuation_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
    return df_c

def tolower_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(tolower)
    return df_c

# DATA Download

In [81]:
FILE_PATH_TED = "ted"
FILE_PATH_TED_S3 = BUCKET + "/" + FILE_PATH_TED

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20230409.csv", mode="rb") as file_in:
    offres_2023_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20221225.csv", mode="rb") as file_in:   
    offres_2022_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/Bulloterie_10_Saison_2.xlsx", mode="rb") as file_in:  
    bulloterie_df =  pd.read_excel(file_in, sheet_name=0,header=[0,1])


/tmp/ipykernel_72898/4280061390.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2023_df = pd.read_csv(file_in, sep=";")
/tmp/ipykernel_72898/4280061390.py:8: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2022_df = pd.read_csv(file_in, sep=";")


In [82]:
with fs.open(BUCKET_CLEM+"/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv", mode="rb") as file_in:
    fdp_dinum_df = pd.read_csv(file_in, sep=";",encoding='Windows-1252')

In [12]:
with fs.open(FILE_PATH_TED_S3+"/dila_refOrga_admin_Etat_fr_20230505.json", mode="rb") as file_in:  
    orga =  json.load(file_in)

### Traitement référentiel services

In [66]:
orga_df = pd.json_normalize(orga,record_path='service')

In [47]:
services_clean = ( orga_df
    .pipe(remove_accent_from_df,['copyright','nom','type_organisme','sigle'])
    .pipe(remove_punctuation_from_df,['copyright','nom','type_organisme','sigle'])
    .pipe(tolower_df,['copyright','nom','type_organisme','sigle'])
)

In [48]:
services_clean = services_clean[['copyright','nom','type_organisme','sigle']]

In [49]:
services_clean.head(5)

,copyright,nom,type_organisme,sigle
0,direction de l information legale et administrative premiere ministre,bureau de coordination strategique,administration centrale ou ministere,
1,direction de l information legale et administrative premiere ministre,direction de l information legale et administrative,administration centrale ou ministere,dila
2,direction de l information legale et administrative premiere ministre,bureau de la simplification et de la qualite de service,administration centrale ou ministere,
3,direction de l information legale et administrative premiere ministre,service de la protection sdlp,administration centrale ou ministere,
4,direction de l information legale et administrative premiere ministre,service national d accueil telephonique de l enfance en danger snated,groupement d interet public,


### Traitement offres

In [14]:
offres_df = pd.concat([offres_2023_df,offres_2022_df])

In [15]:
offres_df.columns = offres_df.columns.map(remove_accent).map(remove_punctuation).map(tolower).map(lambda s: s.replace(' ', '_'))

 offres_df[offres_df['metier'].str.contains('scientist|engineer',na=False)].head(2)

In [17]:
offres_df_clean = ( offres_df
    .pipe(remove_accent_from_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
    .pipe(remove_punctuation_from_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
    .pipe(tolower_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
)

In [67]:
offres_df_clean = offres_df_clean[['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste']]
offres_df_clean.head(2)

,organisme_de_rattachement,metier,intitule_du_poste,localisation_du_poste
0,structures de cooperation territoriale,chargee charge de politique environnementale,geomaticien ne ecologue syndicat mixte de gestion du conservatoire botanique,haute loire
1,structures de cooperation territoriale,chargee charge de politique environnementale,charge e mission flore habitats naturels vegetations agro pastorales,haute loire


### Traitement Fiche de poste DNUM

In [84]:
# Traitement des colonnes
fdp_dinum_df.columns = fdp_dinum_df.columns.map(remove_accent).map(remove_punctuation).map(tolower).map(lambda s: s.replace(' ', '_'))

In [86]:
fdp_clean = ( fdp_dinum_df
    .pipe(remove_accent_from_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
    .pipe(remove_punctuation_from_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
    .pipe(tolower_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
)

In [88]:
fdp_clean = fdp_clean[['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences']]
fdp_clean.head(2)


,poste,intitule_du_service_demandeur_bureau_section,corps_grade,presentation,missions,competences
0,gestionnaire financier,direction interministerielle du numerique secretariat general pole budget achats,secretaire administratif ve b,service de la premiere ministre place sous l autorite de la ministre de la transformation et de la fonction publiques la direction interministerielle du numerique dinum est en charge de la transformation numerique de l etat elle accompagne les ministeres conseille le gouvernement et developpe des services et ressources numeriques partagees pour les citoyens et les agents publics rejoindre la dinum c est prendre part a un collectif de pres de 200 agents reunis autour d une meme ambition creer grace au numerique le service public de demain le secretariat general de la dinum est responsable des fonctions transverses de la dinum ressources humaines communication finances logistique juridique garantissant le bon fonctionnement des services de la dinum et la qualite des conditions de travail des agents le pole budget et achats est plus particulierement en charge d assurer l execution financiere des actes de gestion de mettre en oeuvre l achat public et de contribuer a la performance et a la qualite de l execution des operations budgetaires et financieres l ensemble de ces actions porte aujourd hui sur trois programmes budgetaires et bop distincts 129 352 et 349,missions vous assurez l instruction et l execution financiere des actes de gestion de la dinum activites principales instruire saisir et suivre les actes de gestion demandes d achat services faits etc des differents programmes geres par la dinum au sein du logiciel chorus formulaires produire des restitutions et ou alertes financieres zdep56 zbud01 etc assurer l interface entre les differents acteurs de la chaine financiere services comptables et budgetaires des spm et fournisseurs afin d en garantir la fluidite contribuer a la gestion et au suivi des investissements fiec fies operations d inventaire notamment de la dinum suivre en continu le paiement et la cloture des engagements traitement des restes a payer en fonction de l evolution des priorites fixees a la dinum reorganisation en cours il elle peut etre amene e a prendre en charge d autres taches et projets,savoirs maitrise des regles et processus financiers lolf gbcp commande publique une premiere experience en gestion financiere dans la fonction publique est requise savoir faire solides connaissances de chorus coeur et formulaires connaissance et pratique souhaitees en matiere d investissement fiec fies gestion d inventaire gerer et exploiter les bases de donnees savoir etre autonomie rigueur et sens des responsabilites appetence pour le domaine du numerique solides capacites d organisation et de travail en equipe
1,adjoint e du de la chef fe de departement et responsable de l incubation de l operateur de produits interministeriels,direction interministerielle du numerique dinum departement de l operateur de produits interministeriels,,la direction interministerielle du numerique dinum est un service de la premiere ministre place sous l autorite du ministre de la transformation et de la fonction publiques elle oriente anime soutient et coordonne les actions des administrations de l etat et celles des organismes places sous sa tutelle visant a ameliorer la qualite l efficacite l efficience et la fiabilite des services rendus par le systeme d information et de communication de l etat elle contribue a l ouverture des donnees publiques aux strategies d innovation ouverte et au travail avec les ecosystemes d innovation elle pilote la politique de mutualisation du systeme d information et de communication de l etat et peut intervenir dans la gouvernance des operations de mutualisation ainsi que dans la conception l elaboration et la mise en oeuvre de ces operations enfin elle soutient le developpement des competences dans le domaine du numerique en facilitant l emergence d une filiere professio

# Insertion des données dans Elastic Search

In [59]:
# Paramétrage du client
es_client = connections.create_connection(hosts=['http://elasticsearch-master:9200/'])

In [63]:
# Fontion qui transforme un dataframe en document/index à stocker dans ElasticSearch
def doc_generator(df,index_name):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": index_name,
                "_type": "_doc",
                "_source": document,
            }

In [68]:
# Indexation des offres
helpers.bulk(es_client, doc_generator(offres_df_clean,"offres"))

/tmp/ipykernel_52094/3268320283.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(offres_df_clean))
/tmp/ipykernel_52094/3268320283.py:1: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(es_client, doc_generator(offres_df_clean))


(406755, [])

In [64]:
# Indexation des services
helpers.bulk(es_client, doc_generator(services_clean,"services"))

/tmp/ipykernel_72898/840881214.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(services_clean,"services"))
/tmp/ipykernel_72898/840881214.py:1: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(es_client, doc_generator(services_clean,"services"))


(7636, [])

In [89]:
# Indexation des fiches de poste
helpers.bulk(es_client, doc_generator(fdp_clean,"fiches"))

/tmp/ipykernel_72898/2169517871.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(fdp_clean,"fiches"))
/tmp/ipykernel_72898/2169517871.py:2: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(es_client, doc_generator(fdp_clean,"fiches"))


(30, [])

In [34]:
from elasticsearch_dsl import Search
s = Search(index='offres').query('match',metier='charge')

In [35]:
r=s.execute()

/opt/mamba/lib/python3.11/site-packages/elasticsearch_dsl/search.py:712: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.search(index=self._index, body=self.to_dict(), **self._params).body,


In [36]:
df_results = pd.DataFrame((d.to_dict() for d in s.scan()))

/opt/mamba/lib/python3.11/site-packages/elasticsearch_dsl/search.py:728: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  for hit in scan(es, query=self.to_dict(), index=self._index, **self._params):


In [38]:
len(df_results)

153861

In [56]:
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
#fullsearch = es.search(index = "sirus_2020", # l'index dans lequel on cherche
#                              q = "CPCU - CENTRALE DE BERCY", # notre requête textuelle
#                              size = 1) # taille de l'ensemble les échos souhaités
# debug? unstable behaviour?
fullsearch = es_client.search(index = "offres", # l'index dans lequel on cherche
                       q = "service de la protection sdlp", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/tmp/ipykernel_72898/1249916277.py:6: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch = es_client.search(index = "offres", # l'index dans lequel on cherche


In [57]:
fullsearch

ObjectApiResponse({'took': 38, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 13.541397, 'hits': [{'_index': 'offres', '_type': '_doc', '_id': 'mlyuJYsBWOZ9YleKE8QM', '_score': 13.541397, '_source': {'organisme_de_rattachement': 'ministere de l interieur', 'metier': 'chargee charge de gestion administrative et ou de paie', 'intitule_du_poste': 'sdlp chef du secretariat de la sous direction de la protection des personnes h f', 'localisation_du_poste': 'paris'}}]}})

In [129]:
offres_df_clean[offres_df_clean['Métier'].str.contains("ux designer",na=False)|offres_df_clean['Intitulé du poste'].str.contains("data designer",na=False)][['Métier','Intitulé du poste']].head(5)

,Métier,Intitulé du poste
136654,architecte technique,expert conception et architecture de donnees d...
136668,architecte technique,expert conception et architecture de donnees d...
136672,architecte technique,expert conception et architecture de donnees d...


In [130]:
offres_df_clean[offres_df_clean['Métier'].str.contains("data scientist",na=False)|offres_df_clean['Intitulé du poste'].str.contains("data scientist",na=False)]['Organisme de rattachement'].unique()

array(['Centre Hospitalier Universitaire de Tours',
       "Muséum National d'Histoire Naturelle",
       'Direction générale des finances publiques (DGFiP)',
       "Institut national de l'information géographique et forestière",
       "Ministère de l'éducation nationale et de la jeunesse  -  Ministère de l'enseignement supérieur et de la recherche",
       'Secrétariat Général', 'Administration Centrale MENJ - MESR',
       'Autorité des Marchés Financiers',
       "Agence Régionale de Santé Provence-Alpes-Côte-d'Azur",
       'Agence de Services et de Paiement (ASP)',
       'Structures de coopération territoriale',
       'Direction Interministérielle du Numérique (DINUM)',
       'Agence Nationale de Sécurité du Médicament et des Produits de Santé',
       'Institut de Recherche pour le Développement',
       "Ministère de l'Intérieur",
       'Direction générale des entreprises (DGE)', 'DRSD',
       'Direction des Services Judiciaires',
       "Ministère des Armées - CMG d'Arcu

In [ ]:
offres_df_clean[]